# 2022.6#2

本周进行的学习分为两个部分：基于 softmax 回归的图像分类（单层），随后建立一个仅包括全连接层的多层网络来进行同样的任务。

> 一点说明：在学习路线方面我们并非没有方向，但在之前的报告中没有明确表明。我们的计划是从最简单的结构开始，从单层、多层过渡到多块，在此过程中熟悉各类损失函数、优化方法和理解训练过程。这个过程持续到能独立建立一个卷积神经网络为止，大概在6月底前完成，然后我们开始项目导向的学习和实践。
>
> 为了尽快跟项目接轨，在进行网络搭建时我们会以图像分类任务为主。
>
> 此外，下周开始我们会接触一些文献中公开的遥感图像模型。

### 数学部分
在图像分类任务中，输出结果常使用 one-hot 编码，即对于 n 个需要判断的类别输出一个n维向量，每个维度代表了模型判断为该类别的可能性。虽然此处的图像分类是针对整幅图像的，但也可以用于单个像元。

针对该输出使用的损失函数是交叉熵损失。这是一种基于最大似然法的评价，将损失函数定义为负对数似然：
$$
-\log P(\mathbf{Y} \mid \mathbf{X})=\sum_{i=1}^{n}-\log P\left(\mathbf{y}^{(i)} \mid \mathbf{x}^{(i)}\right)=\sum_{i=1}^{n} l\left(\mathbf{y}^{(i)}, \hat{\mathbf{y}}^{(i)}\right)
$$
能推导得到损失函数为
$$
l(\mathbf{y}, \hat{\mathbf{y}})=-\sum_{j=1}^{q} y_{j} \log \hat{y}_{j}
$$

In [4]:
# 代码部分
import torch
import torchvision
from torch.utils import data
from torchvision import transforms

# 加载数据。这里使用了经典的MNIST数据集。
# batch大小为50，读入后转为Tensor
batch_size = 50
trans = transforms.Compose([transforms.ToTensor()])
mnist_train = torchvision.datasets.FashionMNIST(root='./data', train=True, transform=trans, download=True)
mnist_test = torchvision.datasets.FashionMNIST(root='./data', train=False, transform=trans, download=True)
loader_train = data.DataLoader(mnist_train, batch_size, shuffle=True, num_workers=8)
loader_test = data.DataLoader(mnist_test, batch_size, shuffle=False, num_workers=8)

In [2]:
# 定义模型
# 一层用于展开数据，另一层为softmax运算
# 该数据集每张图片大小为28x28，共10类，因此仿射矩阵为784x10
# 单张图片存储为1x28x28，用Flatten展开为长为784的向量
from torch import nn
net = nn.Sequential(nn.Flatten(1, -1), nn.Linear(784, 10))

# 初始化参数
nn.init.normal_(net[1].weight, std=0.01)

# 使用交叉熵损失
# 这个函数并不如数学上定义的那么简单，隐藏了大量细节，使得我们只需要传入模型计算得到的概率矩阵即可
# 在实际做softmax运算中需要考虑到指数函数的溢出可能，需要使用各种手段避免
loss = nn.CrossEntropyLoss(reduction='none')

# 随机梯度下降，学习率0.1
trainer = torch.optim.SGD(net.parameters(), lr=0.1)

In [3]:
# 进行训练
# 训练过程与上一次一致。多余的代码是为了计算每个周期训练结束后的模型有效性，输出各种度量。

# 10个周期
epoch_nums = 10

def accuracy(y_hat, y): 
    """计算预测正确的数量"""
    y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

def evaluate_accuracy(net, data_iter):  
    """计算当前模型的预测精度"""
    correct, count = 0, 0
    with torch.no_grad():
        for X, y in data_iter:
            correct += accuracy(net(X), y)
            count += y.numel()
    return correct / count

for epoch in range(epoch_nums):
    loss_acc, count = 0, 0 # 损失总和、数据集计数
    for X, y in loader_train:
        l = loss(net(X), y)
        loss_acc += l.sum()
        count += y.numel()
        trainer.zero_grad()
        l.mean().backward()
        trainer.step()
    print(f'第{epoch + 1}个周期，损失为{loss_acc / count:f}，预测精度为{evaluate_accuracy(net, loader_test)}')

第1个周期，损失为0.608028，预测精度为0.8189
第2个周期，损失为0.484256，预测精度为0.8276
第3个周期，损失为0.462866，预测精度为0.831
第4个周期，损失为0.449367，预测精度为0.806
第5个周期，损失为0.439492，预测精度为0.8396
第6个周期，损失为0.432865，预测精度为0.8418
第7个周期，损失为0.427378，预测精度为0.8414
第8个周期，损失为0.423323，预测精度为0.8391
第9个周期，损失为0.420658，预测精度为0.8169
第10个周期，损失为0.419874，预测精度为0.8385


随着周期数增加预测精度在达到一定程度后反而下降，而且非常不稳定。经测试（50个周期），增加训练量也不会带来明显的改变，相对于图像的复杂来说，这种简单的模型显然不能很好地学习特征表达。

现在我们搭建第一个有一定深度的网络。在原来一个输入层和一个输出层的基础上插入中间的全连接层，或被称为隐藏层。首先尝试插入一个隐藏层。激活函数使用最简单的ReLU。

In [11]:
# 上面已经定义了数据迭代器，损失函数不作改变直接复用，因此需要做的只有重新定义网络

h1 = 256    # 第一个隐藏层节点数

net = nn.Sequential(nn.Flatten(),           # 输入层，图像展开为向量
                    nn.Linear(784, h1),     # 隐藏层
                    nn.ReLU(),              # 激活
                    nn.Linear(h1, 10))      # 输出层

# 需要初始化参数的层有多个
def init_weights(layer):
    if type(layer) == nn.Linear:
        nn.init.normal_(layer.weight, std=0.01)
net.apply(init_weights)

trainer = torch.optim.SGD(net.parameters(), lr=0.1)

# 训练过程也完全一致，写成函数
def train(epoch_nums, loader_train, loader_test, net, loss, trainer):
    for epoch in range(epoch_nums):
        loss_acc, count = 0, 0
        for X, y in loader_train:
            l = loss(net(X), y)
            loss_acc += l.sum()
            count += y.numel()
            trainer.zero_grad()
            l.mean().backward()
            trainer.step()
        print(f'第{epoch + 1}个周期，损失为{loss_acc / count:f}，预测精度为{evaluate_accuracy(net, loader_test)}')
train(10, loader_train, loader_test, net, loss, trainer)

第1个周期，损失为0.645036，预测精度为0.8278
第2个周期，损失为0.432445，预测精度为0.8249
第3个周期，损失为0.386978，预测精度为0.8427
第4个周期，损失为0.360641，预测精度为0.861
第5个周期，损失为0.340659，预测精度为0.8719
第6个周期，损失为0.325347，预测精度为0.8667
第7个周期，损失为0.311953，预测精度为0.8722
第8个周期，损失为0.301817，预测精度为0.8654
第9个周期，损失为0.289214，预测精度为0.8676
第10个周期，损失为0.281806，预测精度为0.8776


In [12]:
# 再加入另外两个隐藏层
# 此处三个超参数为随意确定
h1, h2, h3 = 256, 128, 64
net = nn.Sequential(nn.Flatten(),
                    nn.Linear(784, h1),
                    nn.ReLU(),
                    nn.Linear(h1, h2),
                    nn.ReLU(),
                    nn.Linear(h2, h3),
                    nn.ReLU(),
                    nn.Linear(h3, 10))
net.apply(init_weights)

trainer = torch.optim.SGD(net.parameters(), lr=0.1)

train(10, loader_train, loader_test, net, loss, trainer)

第1个周期，损失为2.137360，预测精度为0.4716
第2个周期，损失为0.858522，预测精度为0.7591
第3个周期，损失为0.529881，预测精度为0.8246
第4个周期，损失为0.434721，预测精度为0.8244
第5个周期，损失为0.388862，预测精度为0.8428
第6个周期，损失为0.359802，预测精度为0.8535
第7个周期，损失为0.337452，预测精度为0.8666
第8个周期，损失为0.323444，预测精度为0.8633
第9个周期，损失为0.306651，预测精度为0.8695
第10个周期，损失为0.295061，预测精度为0.8662


Sequential添加网络层的方式非常自然而舒适。现在整个网络运行模式已经比较清晰，但我们还没有引入*块*（从代码的视角来看是*类*）。

我们下周计划接触一些公开的遥感图像分类训练模型，并尝试用在我们的数据上。同时我们也借此机会彻底整理数据并上传。

**有一些实践上的问题。我们注意到四层的全连接层并不比两层的表现有明显提高。但首先要考虑到这个模型的简单性，稳定在85%以上的精度并不算一个很差的表现，因此增加层数也很难更进一步，可能需要超参数上微妙的调整。**